<a href="https://colab.research.google.com/github/LaizaCostaa/Job_recommendation_system/blob/main/job__recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install openai==0.28.0

In [ ]:
import openai

In [ ]:
from google.colab import drive

In [ ]:
# Configurar acesso ao Google Drive
drive.mount('/content/drive')

# Carregar dados das tabelas
talentos_resumo = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/talentos_resumo.xlsx')
talentos_informacoes_gerais = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/talentos_informacoes_gerais.xlsx')
talentos_competencias = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/talentos_competencias.xlsx')
talentos_educacao = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/talentos_educacao.xlsx')
talentos_experiencia = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/talentos_experiencia.xlsx')
vagas_informacoes = pd.read_excel('/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/vagas_informacoes (5).xlsx')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **PRÉ-PROCESSAMENTO**

In [ ]:
# Remover a coluna 'empresa_experiencia' da tabela de experiência e 'instituicao' da tabela de educação
talentos_experiencia_clean = talentos_experiencia.drop(columns=['empresa_experiencia'])
talentos_educacao_clean = talentos_educacao.drop(columns=['instituicao'], errors='ignore')

In [ ]:
# Criar o texto de experiência sem o nome da empresa
experiencia_texto = "\n".join([f"{row['cargo_experiencia']} de {row['inicio']} até {row['termino']}" for idx, row in talentos_experiencia_clean.iterrows()])

In [ ]:
# Preencher valores ausentes com uma string vazia
talentos_resumo.fillna('', inplace=True)
talentos_competencias.fillna('', inplace=True)
talentos_experiencia_clean.fillna('', inplace=True)
talentos_educacao_clean.fillna('', inplace=True)

<ipython-input-38-d672db8cc2ed>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  talentos_experiencia_clean.fillna('', inplace=True)


In [ ]:
# Remover duplicatas com base no id_candidato
talentos_resumo.drop_duplicates(subset='id_candidato', inplace=True)
talentos_competencias.drop_duplicates(subset='id_candidato', inplace=True)
talentos_experiencia_clean.drop_duplicates(subset='id_candidato', inplace=True)
talentos_educacao_clean.drop_duplicates(subset='id_candidato', inplace=True)

In [ ]:
# Converter para minúsculas e remover espaços extras
talentos_resumo['resumo'] = talentos_resumo['resumo'].str.lower().str.strip()
talentos_competencias['competencias'] = talentos_competencias['competencias'].str.lower().str.strip()
talentos_educacao_clean['curso'] = talentos_educacao_clean['curso'].str.lower().str.strip()

In [ ]:
# Converter colunas de data para formato datetime
talentos_experiencia_clean['inicio'] = pd.to_datetime(talentos_experiencia_clean['inicio'])
talentos_experiencia_clean['termino'] = pd.to_datetime(talentos_experiencia_clean['termino'])

In [ ]:
# Calcular tempo de experiência em meses
talentos_experiencia_clean['tempo_total'] = (talentos_experiencia_clean['termino'] - talentos_experiencia_clean['inicio']).dt.days // 30


# **MONTANDO O SISTEMA DE RECOMENDAÇÃO**


In [ ]:
def set_openai_api_key():
    with open('/content/drive/MyDrive/Portfólio/projeto_recrutamento/open_ai_key.txt') as file:
        OPENAI_API_KEY = file.read().strip()
        openai.api_key = OPENAI_API_KEY


set_openai_api_key()

1. Recomendação para 1 candidato (id como variável de entrada)

In [70]:
# Função para recomendação de vaga
def recomendar_vaga_completa(id_candidato):
    # Informações de exemplo
    candidato = talentos_resumo[talentos_resumo['id_candidato'] == id_candidato]
    competencias = talentos_competencias[talentos_competencias['id_candidato'] == id_candidato]
    experiencia = talentos_experiencia_clean[talentos_experiencia_clean['id_candidato'] == id_candidato]
    educacao = talentos_educacao_clean[talentos_educacao_clean['id_candidato'] == id_candidato]

    # Preparar as informações do candidato
    resumo_candidato = candidato['resumo'].values[0]
    palavras_chave = candidato['palavras_chave'].values[0]
    competencias_texto = ", ".join(competencias['competencias'].values)
    experiencia_texto = "\n".join([f"{row['cargo_experiencia']} de {row['inicio']} até {row['termino']}" for idx, row in experiencia.iterrows()])
    educacao_texto = "\n".join([f"{row['grau']} em {row['curso']}" for idx, row in educacao.iterrows()])

    # Gerar o prompt
    prompt = f"""
    Aqui está o perfil de um candidato:

    Resumo: {resumo_candidato}
    Palavras-chave: {palavras_chave}
    Competências: {competencias_texto}
    Experiência:
    {experiencia_texto}
    Educação:
    {educacao_texto}

    Com base nesses detalhes, recomende uma vaga adequada entre as seguintes opções de vagas:
    {vagas_informacoes[['vaga', 'requisitos']].to_string(index=False)}
    """

    # Requisição para a API usando a nova interface de chat
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente que ajuda a recomendar vagas de trabalho."},
            {"role": "user", "content": prompt}
        ]
    )

    # Exibir a resposta
    recomendacao = response['choices'][0]['message']['content']
    return recomendacao


#exemplo de uso
id_candidato = 1  # ID desejado
recomendacao = recomendar_vaga_completa(id_candidato)
print(recomendacao)


Com base nas informações fornecidas sobre o perfil do candidato, a vaga que parece mais adequada é a de "Assistente de RH". O candidato tem experiência na área de recursos humanos, especificamente em recrutamento e seleção, e treinamento e desenvolvimento, que são competências valiosas para essa posição. Além disso, a vaga requer habilidades de gerenciar várias tarefas e prazos eficientemente, que o candidato pode ter adquirido através de sua experiência como Analista de Assuntos Regulatórios Pleno. A vaga também pede alguém que more em SP capital, mas o local de residência do candidato não foi indicado no perfil fornecido, então isso seria necessário verificar.


2. Matching dos 3 primeiros candidatos

In [ ]:
import time

# Função para gerar a tabela de correspondências para os primeiros 3 candidatos para testar
def gerar_tabela_correspondencias(num_candidatos=3):
    correspondencias = []  # Inicializando a lista para armazenar as correspondências

    total_candidatos = min(num_candidatos, len(talentos_informacoes_gerais))  # Limitar ao número de candidatos disponíveis

    for i in range(total_candidatos):  # Loop pelos primeiros 3 candidatos ou o total disponível
        id_candidato = talentos_informacoes_gerais['id_candidato'].iloc[i]
        print(f"Processando candidato {i+1}/{total_candidatos} (ID: {id_candidato})")

        # Chamada à função que recomenda a vaga
        recomendacao = recomendar_vaga_completa(id_candidato)

        # Simulando a extração da vaga compatível e o percentual de compatibilidade da resposta do GPT
        try:
            vaga_compativel = recomendacao.split("Vaga recomendada:")[1].split(".")[0].strip()
            percentual_compatibilidade = recomendacao.split("Percentual de compatibilidade:")[1].split("%")[0].strip()
            justificativa = recomendacao.split("Justificativa:")[1].strip()
        except:
            vaga_compativel = "Não identificada"
            percentual_compatibilidade = "N/A"
            justificativa = recomendacao

        # Acessar o id e cargo atual do candidato
        cargo_candidato = talentos_informacoes_gerais['cargo_candidato'].iloc[i]

        # Adicionar as correspondências para cada candidato na lista
        correspondencias.append({
            "id_candidato": id_candidato,
            "cargo_atual": cargo_candidato,
            "vaga_compativel": vaga_compativel,
            "percentual_compatibilidade": percentual_compatibilidade,
            "justificativa": justificativa
        })

        # Pausar para não sobrecarregar a API
        time.sleep(1)

    # Converter a lista de dicionários em um DataFrame
    correspondencias_df = pd.DataFrame(correspondencias)

    # Retornar o DataFrame final
    return correspondencias_df

# Gerar a tabela com as correspondências para os primeiros 3 candidatos
tabela_correspondencias = gerar_tabela_correspondencias(num_candidatos=3)

Processando candidato 1/3 (ID: 1)
Processando candidato 2/3 (ID: 2)
Processando candidato 3/3 (ID: 3)
   id_candidato                            cargo_atual   vaga_compativel  \
0             1                 Consultora estratégica  Não identificada   
1             2               Agile Master, KMP, PSM-I  Não identificada   
2             3  Especialista em Consultoria de Gestão  Não identificada   

  percentual_compatibilidade  \
0                        N/A   
1                        N/A   
2                        N/A   

                                       justificativa  
0  Baseado no seu perfil, a vaga de "Coordenadora...  
1  Analisei o seu perfil e as vagas disponíveis e...  
2  Olá, com base no seu perfil e nas vagas dispon...  


4. Alimentando os resultados em planilha no drive



In [ ]:
# Caminho para o arquivo existente no Google Drive (substitua pelo caminho correto)
file_path = '/content/drive/MyDrive/Portfólio/sistema_recomendacao_vagas/tabela_correspondencias_candidatos.csv.xlsx'

# Ler a planilha existente
df_existing = pd.read_excel(file_path)

# Exibir a planilha original (opcional)
print("Planilha original:")
print(df_existing.head())

# Gerar a nova tabela de correspondências (por exemplo, os primeiros 3 candidatos)
tabela_correspondencias = gerar_tabela_correspondencias(num_candidatos=3)

# Atualizar os dados existentes com a nova tabela (append ou substituição)
# Se quiser adicionar os novos dados ao final da planilha existente:
df_updated = pd.concat([df_existing, tabela_correspondencias], ignore_index=True)

# Salvar a planilha atualizada de volta no Google Drive
df_updated.to_excel(file_path, index=False)

print("Planilha atualizada com sucesso!")

Planilha original:
   id_candidato                            cargo_atual   vaga_compativel  \
0             1                 Consultora estratégica  Não identificada   
1             2               Agile Master, KMP, PSM-I  Não identificada   
2             3  Especialista em Consultoria de Gestão  Não identificada   

   percentual_compatibilidade  \
0                         NaN   
1                         NaN   
2                         NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

5. Matching para id's específicos